In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split

class LogDataset(Dataset):
    def __init__(self, dataframe):
        self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        log = item['Message']
        inputs = self.tokenizer(log, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
        inputs = {key: value.squeeze(0) for key, value in inputs.items()}
        
        labels = {
            'is_anomal': torch.tensor(item['IsAnomal'], dtype=torch.long),
            'need_attention': torch.tensor(item['NeedAttention'], dtype=torch.long),
            'level': torch.tensor(item['Level'] - 1, dtype=torch.long)  
        }
        return inputs, labels


# Example CSV loading and train/test split
df = pd.read_csv('/content/drive/MyDrive/clasic-httpserver-logs.csv')
df['Level'] = df['Level'].clip(1, 5)  # Ensuring levels are within 1 to 5
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = LogDataset(train_df)
test_dataset = LogDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [ ]:
from transformers import BertModel
import torch.nn as nn

class MultiTaskBERT(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier_is_anomal = nn.Linear(self.bert.config.hidden_size, 2)
        self.classifier_need_attention = nn.Linear(self.bert.config.hidden_size, 2)
        self.classifier_level = nn.Linear(self.bert.config.hidden_size, 5)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        
        is_anomal_logits = self.classifier_is_anomal(pooled_output)
        need_attention_logits = self.classifier_need_attention(pooled_output)
        level_logits = self.classifier_level(pooled_output)
        
        return is_anomal_logits, need_attention_logits, level_logits


In [ ]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from transformers import get_scheduler
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

def train_model(model, train_loader, device):
    model.train()
    optimizer = AdamW(model.parameters(), lr=5e-5)
    loss_fn = CrossEntropyLoss()
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3)

    for epoch in range(3):  # Adjust the number of epochs according to your needs
        for inputs, labels in train_loader:
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            labels = {k: v.to(device) for k, v in labels.items()}

            # Check labels are in the correct range
            assert (labels['is_anomal'].ge(0) & labels['is_anomal'].le(1)).all()
            assert (labels['need_attention'].ge(0) & labels['need_attention'].le(1)).all()
            assert (labels['level'].ge(0) & labels['level'].le(4)).all()

            model.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = (loss_fn(outputs[0], labels['is_anomal']) +
                    loss_fn(outputs[1], labels['need_attention']) +
                    loss_fn(outputs[2], labels['level'])) / 3
            loss.backward()
            optimizer.step()
            scheduler.step()

        print(f"Epoch {epoch+1}: Loss {loss.item()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiTaskBERT().to(device)
train_model(model, train_loader, device)


In [ ]:
import matplotlib.pyplot as plt

def evaluate_model_and_plot(model, test_loader, device):
    model.eval()
    accuracies_is_anomal = []
    accuracies_need_attention = []
    accuracies_level = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            labels = {k: v.to(device) for k, v in labels.items()}

            outputs = model(input_ids, attention_mask)
            _, preds_is_anomal = torch.max(outputs[0], dim=1)
            _, preds_need_attention = torch.max(outputs[1], dim=1)
            _, preds_level = torch.max(outputs[2], dim=1)
            
            accuracy_is_anomal = (preds_is_anomal == labels['is_anomal']).float().mean()
            accuracy_need_attention = (preds_need_attention == labels['need_attention']).float().mean()
            accuracy_level = (preds_level == labels['level']).float().mean()

            accuracies_is_anomal.append(accuracy_is_anomal.item())
            accuracies_need_attention.append(accuracy_need_attention.item())
            accuracies_level.append(accuracy_level.item())



    # Print average accuracies
    print(f"Average Test Accuracy IsAnomal: {sum(accuracies_is_anomal) / len(accuracies_is_anomal):.4f}")
    print(f"Average Test Accuracy NeedAttention: {sum(accuracies_need_attention) / len(accuracies_need_attention):.4f}")
    print(f"Average Test Accuracy Level: {sum(accuracies_level) / len(accuracies_level):.4f}")


In [ ]:
# Assuming test_loader and device are already set up
evaluate_model_and_plot(model, test_loader, device)


###  Training with User Defined Logs

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/user-defined-logs.csv')
df['Level'] = df['Level'].clip(1, 5) 
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = LogDataset(train_df)
test_dataset = LogDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
train_model(model, train_loader, device)


In [ ]:
evaluate_model_and_plot(model, test_loader, device)

In [ ]:
dummy_input_ids = torch.randint(0, 30522, (1, 128), device=device)  # Example input token IDs
dummy_attention_mask = torch.ones(1, 128, device=device)  # Example attention mask

# Trace the model with dummy inputs
traced_model = torch.jit.trace(model, (dummy_input_ids, dummy_attention_mask))

# Save the traced model
traced_model.save("multi_task_bert_large.pt")